$\theta = \{\psi, \sigma_1, \sigma_2\} $

$p_{\theta}(s_{t+1}|s_t, a_t) = \{_{pdf(x_{t+1}, \; (x_t+a_t^0), \; \theta_2) \; \times \; pdf(y_{t+1}, \; (y_t+a_t^1), \; \theta_2) \; if \; x_t >= \theta_0}^{pdf(x_{t+1}, \; (x_t+a_t^0), \; \theta_1) \; \times \; pdf(y_{t+1}, \; (y_t+a_t^1), \; \theta_1) \; if \; x_t < \theta_0}$

In [1]:
import numpy as np
import random

theta = [5, 2, 4]

def transiction(s, a, p=2):
    return np.array(s) + np.array(a) + np.random.normal(0, p, 2)

def pickModel(s, theta):
    psi, *m = theta
    x,_ = s
    return m[int(x >= psi)]

def generate(s, n, theta):   
    for _ in range(n):
        a = random.choice([(1,0), (-1,0), (0,1), (0,-1)])
        yield s, a
        s = transiction(s, a, pickModel(s, theta))

s_0 = (1,1)
O = [(s, a) for (s, a) in generate(s_0, 100, theta)]
O

[((1, 1), (0, 1)),
 (array([-0.34205444,  4.67199512]), (0, 1)),
 (array([2.00900055, 7.3539974 ]), (0, 1)),
 (array([0.91517047, 8.98834602]), (1, 0)),
 (array([-1.06423127,  9.08041599]), (1, 0)),
 (array([3.91629971, 7.77507107]), (0, -1)),
 (array([5.43637829, 6.96886528]), (0, 1)),
 (array([ 2.08855394, 11.49389515]), (0, 1)),
 (array([ 2.00814495, 13.79200844]), (-1, 0)),
 (array([-1.10101556, 12.45066628]), (0, 1)),
 (array([ 0.305633  , 11.75713749]), (1, 0)),
 (array([-1.71662076, 15.03841277]), (0, -1)),
 (array([-0.09157384, 17.70505487]), (1, 0)),
 (array([-2.0504338 , 16.62519688]), (0, -1)),
 (array([-0.0705911 , 14.61297135]), (0, 1)),
 (array([-3.91738156, 13.07321421]), (0, -1)),
 (array([-2.64119135, 13.13820198]), (0, 1)),
 (array([-6.30204161, 17.43672328]), (-1, 0)),
 (array([-7.44224207, 15.64306467]), (0, 1)),
 (array([-7.47401813, 15.84847236]), (0, -1)),
 (array([-7.24300614, 13.06116905]), (1, 0)),
 (array([-7.87186203, 12.05161202]), (-1, 0)),
 (array([-10.60

$p_{\theta}(s_{t+1}|s_t, a_t) = pdf(x_{t+1}, (x_t+a_t^0), \theta_2) \; \times \; pdf(y_{t+1}, (y_t+a_t^1), \theta_2) \; \times \; \mathbb{I}_{x_t >= \theta_0} + pdf(x_{t+1}, (x_t+a_t^0), \theta_1) \; \times \; pdf(y_{t+1}, (y_t+a_t^1), \theta_1) \; \times \; \mathbb{I}_{x_t < \theta_0}$



$p_{\theta}(s_{t+1}|s_t, a_t) = pdf(x_{t+1}, (x_t+a_t^0), \theta_2) \; \times \; pdf(y_{t+1}, (y_t+a_t^1), \theta_2) \; \times \; \frac{1}{1+e^{\beta (\theta_0 - x_t)}}$ <br>
$ \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; $
$ + \; pdf(x_{t+1}, (x_t+a_t^0), \theta_1) \; \times \; pdf(y_{t+1}, (y_t+a_t^1), \theta_1) \; \times \; (1 - \frac{1}{1+e^{\beta (\theta_0 - x_t)}})$ 


$\cal{L}[\xi | \theta] = \sum_{t=0}^{\mathrm{T}-1} log \; p_{\theta}(s_{t+1}|s_t, a_t)$

In [2]:
from scipy.stats import norm

def ll(O, theta, beta=10):
    psi, sig1, sig2 = theta
    # p = lambda s_, s, a: norm.pdf(s_[0], (np.array(s[0]) + np.array(a[0])), m[int(s[0] >= psi)]) * norm.pdf(s_[1], (np.array(s[1]) + np.array(a[1])), m[int(s[0] >= psi)])
    p = lambda x_,y_, x,y, ax,ay: ( 
        norm.pdf(x_, (x + ax), sig2)  
        * norm.pdf(y_, (y + ay), sig2) 
        * (1/(1 + np.e ** (beta * (psi-x))))
        +
        norm.pdf(x_, (x + ax), sig1) 
        * norm.pdf(y_, (y + ay), sig1)
        * (1 - (1/(1 + np.e ** (beta * (psi-x)))))
    )
    return np.sum([np.log(p(*s, *O[t-1][0], *O[t-1][1])) for t,(s,_) in enumerate(O) if 0 < t])

ll(O, (5, 2, 4))

-427.0669422682604

In [3]:
# best_theta = 0
# best_ll = -np.inf

# for psi in range(10):
#     for p1 in range(10):
#         for p2 in range(10):
#             theta = (psi+1, p1+1, p2+1)
#             new_ll = ll(O, theta)
#             if new_ll > best_ll:
#                 best_ll = new_ll
#                 best_theta = theta

# print(best_theta, best_ll)

In [10]:
import torch
from torch.autograd import Variable
from torch.distributions import normal

s = Variable(torch.from_numpy(np.array(O)[:-1,0])).type(torch.FloatTensor)
a = Variable(torch.from_numpy(np.array(O)[:-1,1])).type(torch.IntTensor)
s_ = Variable(torch.from_numpy(np.array(O)[1:,0])).type(torch.FloatTensor)

p = Variable(torch.Tensor([5, 2, 4]), requires_grad=True)

# loglike = torch.sum(normal.Normal(s[:,0] + a[:,0], torch.where(s[:,0] >= p[0], p[2], p[1])).log_prob(s_[:,0]) + normal.Normal(s[:,1] + a[:,1], torch.where(s[:,0] >= p[0], p[2], p[1])).log_prob(s_[:,1]))
loglike = torch.sum(
        torch.log(
        torch.exp(normal.Normal(s[:,0] + a[:,0], p[2]).log_prob(s_[:,0]))
        * torch.exp(normal.Normal(s[:,1] + a[:,1], p[2]).log_prob(s_[:,1]))
        * torch.sigmoid(-10 * (p[0]-s[:,0]))
        + 
        torch.exp(normal.Normal(s[:,0] + a[:,0], p[1]).log_prob(s_[:,0]))
        * torch.exp(normal.Normal(s[:,1] + a[:,1], p[1]).log_prob(s_[:,1]))
        * (1 - torch.sigmoid(-10 * (p[0]-s[:,0])))
    )
)

loglike.backward()
gradiente = p.grad.clone()
print('like: ', loglike)
# gradiente.numpy()

like:  tensor(-427.0669, grad_fn=<SumBackward0>)


array([ 0.48820096,  7.686308  , -0.15903634], dtype=float32)

In [30]:
import torch
from torch.autograd import Variable
from torch.distributions import normal

s = Variable(torch.from_numpy(np.array(O)[:-1,0])).type(torch.FloatTensor)
a = Variable(torch.from_numpy(np.array(O)[:-1,1])).type(torch.IntTensor)
s_ = Variable(torch.from_numpy(np.array(O)[1:,0])).type(torch.FloatTensor)

p = Variable(torch.randn(3), requires_grad=True)
beta = .2

while p[1] <= 0 or p[2] <= 0:
    p = Variable(torch.randn(3), requires_grad=True)

learning_rate = 0.00002
e = np.inf
for t in range(10000):
    print(p)
    
    loglike = torch.sum(
        torch.log(
            torch.exp(normal.Normal(s[:,0] + a[:,0], p[2]).log_prob(s_[:,0]))
            * torch.exp(normal.Normal(s[:,1] + a[:,1], p[2]).log_prob(s_[:,1]))
            * torch.sigmoid(-beta * (p[0]-s[:,0]))
            + 
            torch.exp(normal.Normal(s[:,0] + a[:,0], p[1]).log_prob(s_[:,0]))
            * torch.exp(normal.Normal(s[:,1] + a[:,1], p[1]).log_prob(s_[:,1]))
            * (1 - torch.sigmoid(-beta * (p[0]-s[:,0])))
        )
    )
    
    loglike.backward()
    
    new_p = p.data - learning_rate * p.grad.data
    # print(new_p, p, p.grad.data)
    if torch.sum(new_p - p.data) == 0:
        break
    else:
        if not new_p.isnan().any():
            p.data = new_p
            p.grad.data.zero_()
        else:
            break



print("loglike  =", loglike.data.numpy(), "p =", p.data.numpy())

tensor([0.5564, 0.6907, 0.2307], requires_grad=True)
tensor([0.5563, 0.6437, 0.2308], requires_grad=True)
tensor([0.5562, 0.5846, 0.2309], requires_grad=True)
tensor([0.5561, 0.5043, 0.2311], requires_grad=True)
loglike  = -1784.8802 p = [0.55612355 0.504261   0.23106764]
